In [81]:
import pandas as pd
df1=pd.read_csv(r"C:\Users\HP\Desktop\profeb15\train.csv",index_col="Store")
#df1 = pd.read_csv("../input/retaildataset/sales data-set.csv",index_col="Store")
#stores data_set = pd.read_csv("../input/retaildataset/stores data-set.csv")

In [82]:
data=df1.loc["0":"10"]
data.reset_index(level=0, inplace=True)
k=list(data.columns)
k[-1],k[-2]=k[-2],k[-1]
columns_titles = k
data=data.reindex(columns=columns_titles)

In [83]:
from datetime import datetime as dt
data['Date'] = data['Date'].apply(lambda x: (dt.strptime(x,'%Y-%m-%d').date()).toordinal())

In [84]:
# train=[]
# test=[]
# from datetime import datetime as dt
# for i in data.values:
#     i[-2],i[-1]=i[-1],i[-2]
#     year=(i[2].split('-'))[2]
#     d = dt.strptime(i[2], '%Y-%m-%d').date()
#     i[2]=d.toordinal()
#     if(year=="2012"):
#         test.append(i)
#     else:
#         train.append(i)
# print(test)

# train_data=pd.DataFrame(train,columns=k)
# test_data=pd.DataFrame(test,columns=k)


from sklearn.model_selection import train_test_split
x=data.iloc[:,:4]
y=data.iloc[:,4:]
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

#    random forest regressor

In [85]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import math
score=0
final_reg_obj=0
ran_estimators=1

for i in range(1,300,50):

#creating an object for the model
    reg_obj=RandomForestRegressor(n_estimators=i,random_state=0)

#training the model
    reg_obj.fit(train_x,train_y.values.ravel())

#predicting 
    pred_y=reg_obj.predict(test_x)

#checking score 
    pred_score=r2_score(test_y,pred_y)
    if(pred_score>score):
        final_reg_obj=reg_obj
        score=pred_score
        rn_estimators=i
    else:
        del (reg_obj)
        
print(score)
print(rn_estimators)

0.953634427022679
251


# gradient boosting regressor

In [86]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_squared_error
import math
score=0
final_reg_obj=0
gb_estimators=1

for i in range(1,300,50):

#creating an object for the model
    reg_obj=GradientBoostingRegressor(n_estimators=i,random_state=0)

#training the model
    reg_obj.fit(train_x,train_y.values.ravel())

#predicting 
    pred_y=reg_obj.predict(test_x)

#checking score 
    pred_score=(r2_score(test_y,pred_y))
    if(pred_score>score):
        final_reg_obj=reg_obj
        score=pred_score
        gb_estimators=i
    else:
        del (reg_obj)
        
print(score)
print(gb_estimators)


0.9545540700478137
251


# xgboost

In [87]:
import xgboost as xgb
from sklearn.metrics import r2_score,mean_squared_error
import math
score=0
final_reg_obj=0
xg_estimators=1

data_dmatrix = xgb.DMatrix(data=x,label=y)


for i in range(1,300,50):

#creating an object for the model
    xg_reg = xgb.XGBRegressor(n_estimators=i)

#training the model
    xg_reg.fit(train_x,train_y.values.ravel())

#predicting 
    pred_y=xg_reg.predict(test_x)

#checking score 
    pred_score=(r2_score(test_y,pred_y))
    if(pred_score>score):
        final_reg_obj=xg_reg
        score=pred_score
        xg_estimators=i
    else:
        del (xg_reg)
        
print(score)
print(xg_estimators)

[15:16:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9551455531544593
251


# stacking

In [88]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb


#maintaining a list for all the models
models=list()

#converting data into matrix
data_dmatrix = xgb.DMatrix(data=x,label=y)

#creating xgboostregressor object
xg_reg = xgb.XGBRegressor(n_estimators=xg_estimators)

#creating randomforestregressor object
reg_obj=RandomForestRegressor(n_estimators=rn_estimators,random_state=0)

#creating gradientboosting regressor object
grad_obj=GradientBoostingRegressor(n_estimators=gb_estimators)

#appending all objects in the list
models.append(xg_reg)
models.append(reg_obj)
models.append(grad_obj)

In [89]:
# from sklearn.metrics import mean_squared_error,r2_score
# import math
# predicted_rows=list()
# temp=models

# for i in temp:
#     i.fit(x_train,y_train.values.ravel())
#     out=i.predict(x_test)
#     predicted_rows.append(out)
#     print(math.sqrt(r2_score(y_test,out)))
    

In [90]:
# print(predicted_rows)
# l=predicted_rows
# out=[(l[0][i]+l[1][i])/2 for i in range(len(l[0]))]

In [91]:

# print(math.sqrt(r2_score(test_y,out)))
# print(train_x)
# print(train_y)

# training the models using k splits 

In [92]:
import numpy as np
#maintaining a list for level1 predictions
level1_x=[[],[],[]]

#true values 
level1_y=[]

#for splitting the data into k folds
from sklearn.model_selection import KFold

#splitting the data into 5 splits where 1 split is used as test data
kf = KFold(n_splits =5 , shuffle = True, random_state = 5)
for train_index,test_index in kf.split(train_x):
    kf_xtrain,kf_xtest=train_x.iloc[train_index],train_x.iloc[test_index]
    kf_ytrain,kf_ytest=train_y.iloc[train_index],train_y.iloc[test_index]
    
    # train the every model using the same folds
    for i in range(len(models)):
        models[i].fit(kf_xtrain,kf_ytrain.values.ravel())
        
        #predicting values for k-1 split
        out=models[i].predict(kf_xtest)
        
        out=list(out)
        
        #appending the predicted values to the list 
        level1_x[i].extend(out)
    
    #appending actual true values  
    level1_y.extend(kf_ytest.values)

[15:16:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:16:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [93]:
#transposing the predicted values at level1

import numpy as np
level1_x=np.array(level1_x)
level1_x=level1_x.transpose()

# second level predictions

In [94]:
import numpy as np
from sklearn import linear_model

#converting the data into datframes
level1_train_x=pd.DataFrame(level1_x)
level1_train_y=pd.DataFrame(level1_y)

#reshaping the label y
level1_train_y=level1_train_y.squeeze()

#creating ultimate model at level 2
ultimate_model = linear_model.LinearRegression(normalize='True')

#training the model with level1 predictions
ultimate_model.fit(level1_train_x, level1_train_y) 

##ultimate_model=GradientBoostingRegressor()
##ultimate_model.fit(level1_train_x,level1_train_y)
## from sklearn.svm import SVR
##ultimate_model=SVR(kernel="rbf")
##ultimate_model.fit(level1_train_x,level1_train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize='True')

# predicting values using the stacking model

In [95]:
#list to get predicted values for all the three models
predicted=[[],[],[]]


for i in range(len(models)):
    #predicting and adding the values to the predicted list
    predicted[i].extend(models[i].predict(test_x))

#transpose the predicted values to get (n,3) shape
predicted=np.array(predicted)
predicted=predicted.transpose()

#converting into dataframe
level1_predict_x=pd.DataFrame(predicted)


In [96]:
#predicting values using the level1 predictions as input
out_y=ultimate_model.predict(level1_predict_x)

In [97]:
#r2_score for the predicted values
print(math.sqrt(r2_score(test_y,out_y)))

0.9811404885848599
